# Template generation notebook

In this notebook we generate the yaml templates that will serve as inputs for the many instanciation of our pipeline.
All the templates are saved in the same folder (before being givne as input).
When the pipeline has finished running for a given template, this very template is saved in the output folder.

In [29]:
import os
import yaml
import numpy as np

from utils import check_folder, read_yaml, save_yaml, write

In [30]:
path_to_main = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/code/fMRI/main.py"

In [31]:
subject_dict = {'english': [57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
                    72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 91, 92, 93,
                    94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 108, 109, 110, 113, 114, 115],
                'french':[1]
               }

In [32]:
hrf_list = [
    'spm', # hrf model used in SPM
    'spm + derivative', # SPM model plus its time derivative (2 regressors)
    'spm + derivative + dispersion', # idem, plus dispersion derivative (3 regressors)
    'glover', # this one corresponds to the Glover hrf
    'glover + derivative', # the Glover hrf + time derivative (2 regressors)
    'glover + derivative + dispersion' # idem + dispersion derivative
]


In [33]:
template = {
    'tr': 2.,
    'nb_runs': 9,
    'nb_runs_test': 1,
    'subject': None,
    'scaling_mean': True,
    'scaling_var': True,
    'parallel': False,
    'cuda': True,
    'hrf': None,
    'voxel_wise': True,
    'atlas': 'cort-prob-2mm',
    'seed': 1111,
    'alpha_percentile': 99.9,
    'alpha': None,
    'alpha_min_log_scale': 2,
    'alpha_max_log_scale': 5,
    'nb_alphas': 10,
    'optimizing_criteria': 'R2',
    'encoding_model': 'Ridge()',
    'masker_path': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/global_masker_english",
    'smoothed_masker_path': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/smoothed_global_masker_english",
    'path_to_root': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/",
    'path_to_fmridata': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/fMRI",
    'offset_path': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/wave/english/onsets-offsets",
    'duration_path': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/",
    'language': None,
    'output': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/maps/", # Path to the output folder
    'input': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/", # Path to the folder containing the representations
    'models': [],
    'model_name': None,
}

In [34]:
def get_model_template(
    model_name, 
    layers, 
    hidden_states, 
    attention_heads, 
    heads,
    surname,
    data_compression, 
    ncomponents,
    offset_type,
    duration_type,
    shift_surprisal):
    
    columns_to_retrieve = []
    if hidden_states:
        columns_to_retrieve = ['hidden_state-layer-{}-{}'.format(layer, i) for layer in layers for i in range(1, 769)]
    if attention_heads:
        columns_to_retrieve += ['attention-layer-{}-head-{}-{}'.format(layer, head, i) for layer in layers for head in heads for i in range(1, 65)]
    result = { 'model_name': model_name,
        'columns_to_retrieve': str(columns_to_retrieve),
        'surname': surname,
        'data_compression': data_compression,
        'ncomponents': ncomponents,
        'offset_type': offset_type, # word / word+punctuation / ...,
        'duration_type': duration_type,
        'shift_surprisal': shift_surprisal
      }
    return result
    

In [35]:
language = 'english'
templates_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp/templates/"
sh_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp/shell_commands/"

In [36]:
check_folder(templates_folder)
check_folder(sh_folder)
template['language'] = language

### Determining best hrf kernel

In this part we determine the best hrf kernel to use with the Bert-base-cased model.

In [25]:
job_to_launch_path = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp/jobs.txt"

In [26]:
subject_list = [57, 63, 67, 77] # identified correct subjects
model_name = 'bert-base-cased'
hidden_layer_list = [[1], [6], [8], [9], [10], [12]]
attention_layer_head_list = [[7, 6], [4, 10], [8, 1], [8,2], [6,7], [8, 10], [8, 11], [9, 6]]
command_lines = []
path_to_sh = []

In [27]:
for subject in subject_list:
    for hrf in hrf_list:
        template['subject'] = subject
        template['hrf'] = hrf
        hrf = hrf.replace(' ', '')
        # hidden-states analysis
        for hidden_layers in hidden_layer_list:
            model = get_model_template(model_name, hidden_layers, True, 
                                    False, None, "Bert_hidden-layer-{}".format(hidden_layers),
                                    None, None,
                                    "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_{}_hidden-layer-{}'.format(hrf, subject, model_name, hidden_layers[0])
            yaml_path = os.path.join(templates_folder, '{}_{}_{}_hidden-layer-{}.yml'.format(hrf, subject, model_name, hidden_layers[0]))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_{}_hidden-layer-{}.sh'.format(hrf, subject, model_name, hidden_layers[0])))
        
        # Attention analysis
        for (layer, head) in attention_layer_head_list:
            model = get_model_template(model_name, [layer], False,
                                    True, [head], "Bert_attention-layer-{}_head-{}".format(layer, head),
                                    None, None,
                                    "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_{}_attention-layer-{}_head-{}'.format(hrf, subject, model_name, layer, head)
            yaml_path = os.path.join(templates_folder, '{}_{}_{}_attention-layer-{}_head-{}.yml'.format(hrf, subject, model_name, layer, head))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_{}_attention-layer-{}_head-{}.sh'.format(hrf, subject, model_name, layer, head)))
            

In [28]:
for index, command in enumerate(command_lines):
    write(path_to_sh[index], command)
    queue = 'Nspin_long' # 'Nspin_bigM'
    walltime = '99:00:00'
    output_log = '/home/ap259944/logs/log_o_{}'.format(index)
    error_log = '/home/ap259944/logs/log_e_{}'.format(index)
    job_name = os.path.basename(path_to_sh[index]).split('.')[0]
    write(job_to_launch_path, f"qsub -q {queue} -N {job_name} -l walltime={walltime} -o {output_log} -e {error_log} {path_to_sh[index]}")
    

I found that the 'spm + derivative' works best.

### Analysis of Bert bucket versus Sequential

In this part we use Bert-base-cased model and fit a regression model for each layer hidden-states or attention heads for classical sentence-level prediction and sequential prediction. 

In [37]:
job_to_launch_path = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp/jobs.txt"

In [38]:
hrf = 'spm + derivative'

In [39]:
model_names = ['bert-base-cased', 'bert-base-cased_sequential']
hidden_layer_list = [[i] for i in range(13)]
attention_layer_list = [[i] for i in range(1, 13)]
heads = np.arange(1, 13)
attention_layer_head_list = [[7, 6], [4, 10], [8, 1], [8,2], [6,7], [8, 10], [8, 11], [9, 6]]
command_lines = []
path_to_sh = []

In [40]:
for model_name in model_names:
    for subject in subject_dict[language]:
        template['hrf'] = hrf
        template['subject'] = subject
        
        # hidden layers comparison
        for hidden_layers in hidden_layer_list:
            model = get_model_template(model_name, hidden_layers, True, 
                                            False, None, "{}_hidden-layer-{}".format(model_name, hidden_layers),
                                            None, None,
                                            "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_hidden-layer-{}'.format(model_name, subject, hidden_layers[0])
            yaml_path = os.path.join(templates_folder, '{}_{}_hidden-layer-{}.yml'.format(model_name, subject, hidden_layers[0]))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_hidden-layer-{}.sh'.format(model_name, subject, hidden_layers[0])))
        
        # attention layers comparison
        for attention_layers in attention_layer_list:
            model = get_model_template(model_name, attention_layers, False, 
                                            True, heads, "{}_attention-layer-{}".format(model_name, attention_layers),
                                            None, None,
                                            "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_attention_layer-{}'.format(model_name, subject, attention_layers[0])
            yaml_path = os.path.join(templates_folder, '{}_{}_attention-layer-{}.yml'.format(model_name, subject, attention_layers[0]))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_attention-layer-{}.sh'.format(model_name, subject, attention_layers[0])))
            
        # specific attention heads comparison    
        for (layer, head) in attention_layer_head_list:
            model = get_model_template(model_name, [layer], False,
                                    True, [head], "Bert_attention-layer-{}_head-{}".format(layer, head),
                                    None, None,
                                    "word+punctuation", None, False)
            template['models'] = [model]
            template['model_name'] = '{}_{}_attention-layer-{}_head-{}'.format(model_name, subject, layer, head)
            yaml_path = os.path.join(templates_folder, '{}_{}_attention-layer-{}_head-{}.yml'.format(model_name, subject, layer, head))
            save_yaml(template, yaml_path)
            command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
            path_to_sh.append(os.path.join(sh_folder, '{}_{}_attention-layer-{}_head-{}.sh'.format(model_name, subject, layer, head)))
            

In [41]:
for index, command in enumerate(command_lines):
    write(path_to_sh[index], command)
    queue = 'Nspin_long' # 'Nspin_bigM'
    walltime = '99:00:00'
    output_log = '/home/ap259944/logs/log_o_{}'.format(index)
    error_log = '/home/ap259944/logs/log_e_{}'.format(index)
    job_name = os.path.basename(path_to_sh[index]).split('.')[0]
    write(job_to_launch_path, f"qsub -q {queue} -N {job_name} -l walltime={walltime} -o {output_log} -e {error_log} {path_to_sh[index]}")
    

In [28]:
len(command_lines)

3366

### Attention heads analysis 

In this part, we look at the regression model resulting from specific attention heads.

attention_layer_head_list = [[7, 6], [4, 10], [8, 1], [8,2], [6,7], [8, 10], [8, 11], [9, 6]]

Already included in the first comparison...

### Analysis of fine-tuned Bert models

In this part, we fit each layer hidden-states (and potentially specific heads attention) of each fine-tuned Bert model to the fMRI data of all english subjects.